In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import dsbmm_bp.simulation as dsbmm_sim


In [94]:
Q = 4
T = 5
N = 100
Z_1 = np.random.randint(0, Q, size=(N,))
meta_types = ["poisson", "indep bernoulli"]
L = 5
meta_params = [np.random.randint(5, 15, size=(1, Q, T)), np.random.rand(L, Q, T)]

dsbmm_dict = dsbmm_sim.sample_dynsbm_meta(
    Z_1=Z_1, Q=Q, T=T, meta_types=meta_types, meta_dims=[1, L], meta_params=meta_params
)



In [105]:
A = dsbmm_dict['A']

In [109]:
np.array(list(map(np.count_nonzero, A.transpose(2, 0, 1))))

array([1934, 1633, 1576, 1493, 1385])

In [113]:
from numba import typeof,float32
from numba.typed import List 
from numba.types import ListType
typeof([[np.array([1,2])]])
tmp = List.empty_list(ListType(float32[:]))

In [122]:
a = np.ones((10,4))
b = np.ones((10,4))
np.outer(a,b).shape

(40, 40)

In [11]:
test = np.random.rand(5,4,3,3,2)
test /= test.sum(axis=3)[:,:,:,np.newaxis,:]
test.sum(axis=3)[0,0,:,0]

array([1., 1., 1.])

In [8]:
test = np.random.rand(5,4)
test /= test.sum(axis=1)[:,np.newaxis]
test.sum(axis=1)

array([1., 1., 1., 1., 1.])

In [22]:
from numba import njit  

test_marg = np.random.rand(10,5,4)
test_beta = np.random.rand(4,4,5)
np.allclose((test_beta.transpose(1,0,2) * test_marg.sum(axis=0).T).sum(axis=1),np.einsum("itr,rqt->qt", test_marg, test_beta))
# test = np.random.rand(10,10,3) > 0.2
# @njit
# def test_fun(test):
#     def get_neighbours(A):
#         edge_locs = np.array(list(zip(*A.nonzero())))
#         return edge_locs
#     edge_locs = get_neighbours(test)
#     for i,j,t in edge_locs:
#         print(i,j,t) 
# test_fun(test)

True